In [1]:
%pip install pytorch-pretrained-bert

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install 'keras<3.0.0' mediapipe-model-maker

Note: you may need to restart the kernel to use updated packages.


The system cannot find the file specified.


In [4]:
import torch
import torch.nn as nn
import torchvision.utils as vutils
import torchvision.transforms as transforms
import os
import sys
input_dir = r'D:\clg files\RecSys\py_files'
sys.path.append(input_dir)

from data_util import AttDesDataset
from utils import weights_init

# from dcgan_model import Generator, Discriminator
import time
import imageio
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

c:\Users\Yashk\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Setting device to cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using Device", device)

Using Device cpu


In [6]:
# directory to store output images
output_save_path = './generated_images/'
os.makedirs(output_save_path, exist_ok=True)

In [7]:
# directory to store trained models
model_save_path = './saved_models/'
os.makedirs(model_save_path, exist_ok=True)

In [8]:
from torch.utils.data import Dataset

In [9]:
from torch.nn.utils.rnn import pad_sequence

class CustomDataset(Dataset):
    def __init__(self, data):
        self.images = data['images']
        self.captions = data['captions']    
        self.wrong_images = data['wrong_images']

    def __getitem__(self, index):
        image = self.images[index]
        caption = self.captions[index]
        caption_tensor = torch.tensor(caption)
        wrong_images = self.wrong_images[index] 
        return image, caption_tensor, wrong_images

    def __len__(self):
        return len(self.images)

In [10]:
def collate_fn(batch):
    images = [item[0] for item in batch]
    captions = [item[1] for item in batch]

    # Stack images
    images = torch.stack(images, 0)

    # Truncate or pad captions
    max_length = 128  # whatever fixed length
    captions = [cap[:max_length] for cap in captions]  # Truncate
    embedding_size = 768 
    captions = [torch.cat([cap, torch.zeros(max_length - cap.size(0), embedding_size)]) for cap in captions]

    # Stack captions
    captions = torch.stack(captions, 0)

    return images, captions

In [11]:
from PIL import Image

directory = r'D:\clg files\RecSys\data_files\wrong_images'
normalize = transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
transform = transforms.Compose([
                                            transforms.Resize((448,448)),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            normalize,
                                        ])
print(directory)
wrong_imgs = []
for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
                full_path = os.path.join(directory, filename)
                img = Image.open(full_path)
                img = transform(img)
                wrong_imgs.append(img)

D:\clg files\RecSys\data_files\wrong_images


In [12]:
print(len(wrong_imgs))

488


In [13]:
wrong_imgs[0].shape

torch.Size([3, 448, 448])

In [14]:
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

data_root = r'data_files\smoll.csv'
split_root = ''
dataset_name = 'Furniture'
normalize = transforms.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
batch_size = 4 #128
dataset = AttDesDataset(data_root, dataset_name, transform=transforms.Compose([
                                            transforms.Resize((448,448)),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            normalize,
                                        ]))

temp_dataset = {'images': dataset.images, 'captions': dataset.descriptions, 'wrong_images': wrong_imgs}
custom_dataset = CustomDataset(temp_dataset)
print(len(custom_dataset))
# print(len(custom_dataset['captions']))
num_samples = len(custom_dataset)
indices = list(range(num_samples))
split = int(np.floor(0.8 * num_samples))  # 80-20 train-validation split

# Randomly shuffle the indices
np.random.shuffle(indices)

# Split the indices into train and validation sets
train_indices, val_indices = indices[:split], indices[split:]

# Define samplers for train and validation sets
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

# Define DataLoader for train and validation sets using samplers
train_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(custom_dataset, batch_size=batch_size, sampler=val_sampler, num_workers=0)

# Check the number of batches in train and validation loaders
print("No of batches in train loader: ", len(train_loader))
print("No of batches in validation loader: ", len(val_loader))



440
No of batches in train loader:  110
No of batches in validation loader:  22


In [15]:
print(len(wrong_imgs))

488


In [16]:
len(dataset.images)

440

In [17]:
len(dataset.descriptions)

440

In [18]:
dataset.images[0].shape

torch.Size([3, 448, 448])

In [19]:
len(dataset.descriptions[0])

100

In [20]:
print(dataset.descriptions[0])

tensor([[ 0.0325,  0.0310, -0.2749,  ...,  0.1786, -0.0305, -0.0821],
        [ 0.3724,  1.5462,  0.4730,  ..., -1.5460, -1.3778, -0.4613],
        [ 0.9117, -0.3610,  0.1527,  ..., -0.8065,  1.4866,  0.3510],
        ...,
        [ 0.2698,  0.7917, -0.7626,  ...,  0.6083, -0.4508, -0.0309],
        [-0.5603,  0.8609, -1.1632,  ...,  1.6056, -0.3746, -0.0662],
        [-0.4179,  0.0190, -0.6336,  ...,  1.9185, -1.0130,  0.8874]])


In [21]:
print(len(temp_dataset['captions'][0]))

100


In [22]:
len(custom_dataset[1][1])

100

In [23]:
# for i, data in enumerate(train_loader):
#     print(type(data))  # This will print the type of the data element
#     if isinstance(data, list) or isinstance(data, tuple):
#         print(len(data))  
#     elif isinstance(data, dict):
#         print(data.keys())
#     if i >= 0: 
#         break

In [24]:
prev1 = 448

for batch_idx,batch in enumerate(train_loader):
    print(batch_idx)
    print(type(batch))
    print(batch[0].shape)
    print(batch[1].shape)
    if batch_idx == 0:
        break   
    # if(batch[0].shape[2] != prev1 and batch[3].shape[0] != prev1):
    #     print("Batch size mismatch")
    #     print(f'batch_idx: {batch_idx}, batch[0].shape: {batch[0].shape}, batch[1].shape: {batch[1].shape}')
    #     break

0
<class 'list'>
torch.Size([4, 3, 448, 448])
torch.Size([4, 100, 768])


In [25]:
print(len(custom_dataset.captions[0]))

100


In [26]:
# setting up parameters
noise_dim = 100
embed_dim = 768
embed_out_dim = 100
batch_size = 4 #128
real_label = 1
fake_label = 0
learning_rate = 0.0002
l1_coef = 50
l2_coef = 100

num_epochs = 10
log_interval = 18 #43

In [27]:
# loss functions
criterion = nn.BCELoss()
l2_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

In [28]:
# lists to store losses
D_losses = []
G_losses = []

In [29]:
# initializing generator
from dcgan1 import Generator, Discriminator

generator = Generator(channels=3, embed_dim=embed_dim, noise_dim=noise_dim, embed_out_dim=embed_out_dim).to(device)
generator.apply(weights_init)

Generator(
  (text_embedding): Sequential(
    (0): Linear(in_features=768, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (model): Sequential(
    (0): ConvTranspose2d(10100, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1

In [30]:
# initializing discriminator
discriminator = Discriminator(channels=3, embed_dim=embed_dim, embed_out_dim=embed_out_dim).to(device)
discriminator.apply(weights_init)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (text_embedding): Embedding(
    (text_embedding): Sequential(
      (0): Linear(in_features=768, out_features=100, bias=True)
      (1): BatchNorm1d(100, eps=1e-05, mom

In [31]:
# setting up Adam optimizer for Generator and Discriminator
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(0.5, 0.999))

In [32]:
# training loop
import torch.nn.functional as F
import PIL
# iterating over number of epochs
from datetime import date
from datetime import timedelta
start_time = time.time()
for epoch in range(num_epochs):
    
    batch_time = time.time()
    print('Epoch: {}'.format(epoch+1))
    #iterating over each batch
    for batch_idx,batch in enumerate(train_loader):   

        print(f'epoch: {epoch+1}, batch: {batch_idx+1}/{len(train_loader)}')
        # print(batch_idx)
        
        # reading the data into variables and moving them to device
        images = batch[0].to(device)
        # print('hi after images')
        wrong_images = batch[2].to(device)
        # print('hi after wrong images')
        embeddings = batch[1].to(device).float()

        batch_size = images.size(0)
        # print(f'embeddings size: {embeddings.size()}')
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        # print('training the descriminator')
        # Clear gradients for the discriminator
        optimizer_D.zero_grad()
        
        # Generate random noise
        noise_dim = 100  
        batch_size = images.size(0)
        noise = torch.randn(embeddings.size(0), noise_dim).to(device)
        # noise = noise.view(embeddings.size(0), noise_dim, 1, 1)
   
        # Generate fake image batch with the generator
        # print(f'noise size: {noise.size()}')
        # print(f'embeddings size: {embeddings.size()}')
        
        fake_images = generator(noise, embeddings)
        # print('hi after fake images')
        # if batch_idx == 0:
            # print(f'noise size: {noise.size()}')
            # print(f'embeddings size: {embeddings.size()}')
            # print(f'fake_images size: {fake_images.size()}')
        fake_images = F.interpolate(fake_images, size=(448, 448), mode='bilinear', align_corners=False)
        # print('fake images after interpolation: ', fake_images.size())
        # Forward pass real batch and calculate loss

        real_out, real_act = discriminator(images, embeddings)
        d_loss_real = criterion(real_out, torch.full_like(real_out, real_label, device=device))
        
        # Forward pass wrong batch and calculate loss
        wrong_out, wrong_act = discriminator(wrong_images, embeddings)
        d_loss_wrong = criterion(wrong_out, torch.full_like(wrong_out, fake_label, device=device))
        
        # Forward pass fake batch and calculate loss
        fake_out, fake_act = discriminator(fake_images.detach(), embeddings)
        d_loss_fake = criterion(fake_out, torch.full_like(fake_out, fake_label, device=device))
        
        # Compute total discriminator loss
        d_loss = d_loss_real + d_loss_wrong + d_loss_fake
        
        # Backpropagate the gradients
        d_loss.backward()
        
        # Update the discriminator
        optimizer_D.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #
        # print('Training the generator')
        # Clear gradients for the generator
        optimizer_G.zero_grad()
       
        # Generate new fake images using Generator
        fake_images = generator(noise, embeddings)
        # print(f'fake_images size: {fake_images.size()}')
        fake_images = F.interpolate(fake_images, size=(448, 448), mode='bilinear', align_corners=False)
        # print('fake images after interpolation: ', fake_images.size())
        # Get discriminator output for the new fake images
        out_fake, act_fake = discriminator(fake_images, embeddings)
        # Get discriminator output for the real images
        out_real, act_real = discriminator(images, embeddings)
        
        # Calculate losses
        g_bce = criterion(out_fake, torch.full_like(out_fake, real_label, device=device)) 
        g_l1 = l1_coef * l1_loss(fake_images, images)
        g_l2 = l2_coef * l2_loss(torch.mean(act_fake, 0), torch.mean(act_real, 0).detach())
        
        # Compute total generator loss
        g_loss = g_bce + g_l1 + g_l2
        
        # Backpropagate the gradients
        g_loss.backward()
        
        # Update the generator
        optimizer_G.step()
        
        # adding loss to the list
        D_losses.append(d_loss.item())
        G_losses.append(g_loss.item())
        
        # progress based on log_interval
        if (batch_idx+1) % log_interval == 0 and batch_idx > 0:
            print('Epoch {} [{}/{}] loss_D: {:.4f} loss_G: {:.4f} time: {:.2f}'.format(
                          epoch+1, batch_idx+1, len(train_loader),
                          d_loss.mean().item(),
                          g_loss.mean().item(),
                          time.time() - batch_time))
        
        # storing generator output after every 10 epochs
        if batch_idx == len(train_loader)-1 and ((epoch+1)%10==0 or epoch==0):
            viz_sample = torch.cat((images[:32], fake_images[:32]), 0)
            current_time = datetime.now().strftime("%H_%M_%S")
            vutils.save_image(viz_sample,
            os.path.join(output_save_path, 'output_{}_epoch_{}_time_{}.png'.format(date.today(),epoch+1, current_time)),
                            nrow=8,normalize=True)
            # viz_sample.show()


total_time = time.time() - start_time
formatted_time = str(timedelta(seconds=total_time))

print('Total train time: {}'.format(formatted_time))

Epoch: 1
epoch: 1, batch: 1/110
epoch: 1, batch: 2/110
epoch: 1, batch: 3/110
epoch: 1, batch: 4/110
epoch: 1, batch: 5/110
epoch: 1, batch: 6/110
epoch: 1, batch: 7/110
epoch: 1, batch: 8/110
epoch: 1, batch: 9/110
epoch: 1, batch: 10/110
epoch: 1, batch: 11/110
epoch: 1, batch: 12/110
epoch: 1, batch: 13/110
epoch: 1, batch: 14/110
epoch: 1, batch: 15/110
epoch: 1, batch: 16/110
epoch: 1, batch: 17/110
epoch: 1, batch: 18/110
Epoch 1 [18/110] loss_D: 1.6885 loss_G: 49.5569 time: 84.18
epoch: 1, batch: 19/110
epoch: 1, batch: 20/110
epoch: 1, batch: 21/110
epoch: 1, batch: 22/110
epoch: 1, batch: 23/110
epoch: 1, batch: 24/110
epoch: 1, batch: 25/110
epoch: 1, batch: 26/110
epoch: 1, batch: 27/110
epoch: 1, batch: 28/110
epoch: 1, batch: 29/110
epoch: 1, batch: 30/110
epoch: 1, batch: 31/110
epoch: 1, batch: 32/110
epoch: 1, batch: 33/110
epoch: 1, batch: 34/110
epoch: 1, batch: 35/110


In [ ]:
# saving the trained models
torch.save(generator.state_dict(), os.path.join(model_save_path, 'generator_{}.pth'.format(date.today(),epoch+1)))
torch.save(discriminator.state_dict(), os.path.join(model_save_path,'discriminator_{}.pth'.format(date.today(),epoch+1)))

In [ ]:
# generator loss plot
plt.figure(figsize=(10,5))
plt.title("Generator Loss During Training")
plt.plot(G_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.show()

plt.savefig(os.path.join(output_save_path, 'output_generatorLoss_{}.png'.format(date.today())))

In [ ]:
# discriminator loss plot
plt.figure(figsize=(10,5))
plt.title("Discriminator Loss During Training")
plt.plot(D_losses)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.show()

plt.savefig(os.path.join(output_save_path, 'output_discriminatorLoss_{}.png'.format(date.today())))

In [ ]:
import torchvision.utils as vutils
import os
from datetime import date

# Assuming output_save_path and epoch are defined
output_save_path = './'
epoch = 1

vutils.save_image(viz_sample, os.path.join(output_save_path, 'output_{}_epoch_{}.png'.format(date.today(), epoch+1)), normalize=True)

In [ ]:
viz_sample = torch.cat((images[:32], fake_images[:32]), 0)
vutils.save_image(viz_sample,
                  os.path.join(output_save_path, 'output_{}_epoch_{}.png'.format(date.today(),epoch+1)),
                              nrow=8,normalize=True)
# viz_sample.show()

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Assuming viz_sample is your tensor
# First, detach and move the tensor to cpu
viz_sample = viz_sample.detach().cpu()



In [ ]:
# Convert the tensor to a PIL Image
first_image = viz_sample[0]
viz_sample_pil = transforms.ToPILImage()(first_image)

# Display the image
viz_sample_pil.show()

In [ ]:
# Convert the tensor to a PIL Image
first_image = viz_sample[1]
viz_sample_pil = transforms.ToPILImage()(first_image)

# Display the image
viz_sample_pil.show()

In [ ]:
# Convert the tensor to a PIL Image
first_image = viz_sample[2]
viz_sample_pil = transforms.ToPILImage()(first_image)

# Display the image
viz_sample_pil.show()

In [ ]:
# Convert the tensor to a PIL Image
first_image = viz_sample[3]
viz_sample_pil = transforms.ToPILImage()(first_image)

# Display the image
viz_sample_pil.show()

In [ ]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel

def get_embeddings(text):
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    bert_model = BertModel.from_pretrained('bert-base-chinese')
    des_len = 100
    tokens = ["[CLS]"]
    token_obj = tokenizer.tokenize(text)
    tokens.extend(token_obj)
    tokens.append("[SEP]")
    tokens = tokenizer.convert_tokens_to_ids(tokens)

        # Pad or truncate to the desired length
    if len(tokens) < des_len:
        tokens += [0] * (des_len - len(tokens))  # Pad with zeros
    else:
        tokens = tokens[:des_len]  # Truncate to the maximum length
    

    input_ids = tokens
    input_ids = torch.tensor(input_ids).unsqueeze(0)

     # Get BERT outputs
    with torch.no_grad():
                
        input_ids = torch.tensor(input_ids)
        # print(type(input_ids))
        outputs = bert_model(input_ids)
            
    output_tensor = outputs[0]
    # Get the vector of the [CLS] token (first token)
    cls_vector = output_tensor[0][0][:]
    return cls_vector
 

In [ ]:

def generate_image(caption):
    # Convert caption to embedding (if needed) and generate noise vector
    # You need to define how to convert the caption to an embedding
    # For example, you can use a pre-trained language model like BERT to get embeddings
    
    # Generate fake image using Generator
    with torch.no_grad():
        # Assuming you have a function to convert caption to embeddings
        embeddings = get_embeddings(caption)
        print(f'embeddings size: {embeddings.size()}')
        noise = torch.randn(batch_size, embeddings.size(0))
        print(f'noise size: {noise.size()}')
        embeddings = embeddings.unsqueeze(0)
        embeddings = embeddings.repeat(batch_size, 1, 1)
        fake_image = generator(noise, embeddings)
        fake_image = F.interpolate(fake_image, size=(448, 448), mode='bilinear', align_corners=False)
    
    return fake_image

In [ ]:
from translate import Translator

def translate_to_chinese(text):
    translator = Translator(to_lang="zh")
    translation = translator.translate(text)
    return translation


In [ ]:
caption = "A black room with no furniture"
chinese_caption = translate_to_chinese(caption)
print(f'chinese_caption: {chinese_caption}')
generated_image = generate_image(chinese_caption)   
transforms.ToPILImage()(generated_image[1]).show()
